In [7]:
from textblob import TextBlob
import pandas as pd


In [8]:
df_09= pd.read_pickle(r"C:\Users\danie\Documents\GitHub\Masters-Thesis\2009_preprocessed_date.pickle") 

In [9]:
def subjectivity(sentence):
    subjectivity = ""

    subjectivity = TextBlob(sentence).sentiment.subjectivity

    return subjectivity

def polarity(sentence):
    polarity = ""

    polarity = TextBlob(sentence).sentiment.polarity

    return polarity

In [10]:
df_09['subjectivity'] = df_09['sentences'].apply(subjectivity)
df_09['polarity'] = df_09['sentences'].apply(polarity)



In [19]:
df_09.head(5)

,sentences,article_id,year,encoded_sentences,subjectivity,polarity
91571,Liam Gallagher has broken the silence surround...,5048,2009,"([break, silence, surround, ', break, say, ban...",0.35,-0.2
91572,"However, in an interview with The Times Liam G...",5048,2009,"([however, interview, the, say, longer], 2, 0, 0)",0.00,0.0
91573,I think we all know that.,5048,2009,"([i, think, know], 0, 0, 0)",0.00,0.0
91574,So that's done.,5048,2009,"([so], 0, 0, 0)",0.00,0.0
91575,"""""It's a shame, but that's life.",5048,2009,"([shame, life], 0, 0, 0)",0.00,0.0


In [20]:
#LSTM Sentiemnt Analysis 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

X = df_09['sentences']
y = df_09['subjectivity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)

# Tokenize the data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

# Convert the texts to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure equal length
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=maxlen))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
243/243 [==============================] - 16s 53ms/step - loss: 0.6470 - accuracy: 0.2847 - val_loss: 0.6444 - val_accuracy: 0.2851
Epoch 2/10
243/243 [==============================] - 10s 40ms/step - loss: 0.6456 - accuracy: 0.2859 - val_loss: 0.6418 - val_accuracy: 0.2851
Epoch 3/10
243/243 [==============================] - 10s 42ms/step - loss: 0.6449 - accuracy: 0.2859 - val_loss: 0.6417 - val_accuracy: 0.2851
Epoch 4/10
243/243 [==============================] - 10s 42ms/step - loss: 0.6450 - accuracy: 0.2859 - val_loss: 0.6427 - val_accuracy: 0.2851
Epoch 5/10
243/243 [==============================] - 10s 42ms/step - loss: 0.6456 - accuracy: 0.2859 - val_loss: 0.6425 - val_accuracy: 0.2851
Epoch 6/10
243/243 [==============================] - 10s 41ms/step - loss: 0.6450 - accuracy: 0.2859 - val_loss: 0.6417 - val_accuracy: 0.2851
Epoch 7/10
243/243 [==============================] - 10s 40ms/step - loss: 0.6452 - accuracy: 0.2859 - val_loss: 0.6421 - val_accuracy: